In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

In [ ]:
# Read the file, treat the first row as header, ignore leading "1" index in second row
target_df = pd.read_csv('./data/KIRC_data/KIDNEY_RANDOM_TARGET.txt',
    sep=r'\s+',         # use whitespace as separator
    header=0,                      # first row contains column names (sample IDs)
    quotechar='"',                 # handle quoted sample IDs
)
target_df.head()

In [ ]:
# Read the file, treat the first row as header, ignore leading "1" index in second row
methyl_df = pd.read_csv('./data/KIRC_data/KIDNEY_RANDOM_Methy_FEATURES.txt',
    sep=r'\s+',         # use whitespace as separator
    header=0,                      # first row contains column names (sample IDs)
    quotechar='"',                 # handle quoted sample IDs
)
print(methyl_df.shape)
print("Number of missing values:", methyl_df.isnull().sum().sum())
methyl_df.head()

In [ ]:
# Read the file, treat the first row as header, ignore leading "1" index in second row
ge_df = pd.read_csv('./data/KIRC_data/KIDNEY_RANDOM_mRNA_FEATURES.txt',
    sep=r'\s+',         # use whitespace as separator
    header=0,                      # first row contains column names (sample IDs)
    quotechar='"',                 # handle quoted sample IDs
)
print(ge_df.shape)
print("Number of missing values:", ge_df.isnull().sum().sum())
ge_df.head()

## Training Random Forest on Methylation data

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
target_series = target_df.iloc[0]

# Reindex X and y to ensure alignment by sample ID
# The files has been matched already, but we do it here for educational purposes.
target_series = target_df.iloc[0]
common_samples = methyl_df.index.intersection(target_series.index)
methyl_df_matched = methyl_df.loc[common_samples]
y = target_series.loc[common_samples].astype(int)
print(y.sum())

# ---  Impute missing values ---
imputer = SimpleImputer(strategy='mean')
X_methyl_imputed = imputer.fit_transform(methyl_df_matched)

sss = StratifiedShuffleSplit(n_splits=30, test_size=0.3, random_state=42)
accuracies_meth = []

for train_index, test_index in sss.split(X_methyl_imputed, y):
    X_train, X_test = X_methyl_imputed[train_index], X_methyl_imputed[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    accuracies_meth.append(acc)

# --- Plot ---
plt.figure(figsize=(8, 5))
plt.boxplot(accuracies_meth)
plt.title("Stratified Random Forest Accuracy (30 Splits)")
plt.ylabel("Accuracy")
plt.xticks([1], ["Random Forest"])
plt.grid(True)
plt.show()

## Training Random Forest on Gene Expression data

In [ ]:
# Reindex X and y to ensure alignment by sample ID
# The files has been matched already, but we do it here for educational purposes.
target_series = target_df.iloc[0]
common_samples = ge_df.index.intersection(target_series.index)
ge_df_matched = ge_df.loc[common_samples]
y = target_series.loc[common_samples].astype(int)
X_ge = ge_df_matched.to_numpy()
# print(y.sum())

# --- Impute missing values ---
# imputer = SimpleImputer(strategy='mean')
# X_ge_imputed = imputer.fit_transform(ge_df_matched)

 
sss = StratifiedShuffleSplit(n_splits=30, test_size=0.3, random_state=42)
accuracies_ge = []

for train_index, test_index in sss.split(X_ge, y):
    X_train, X_test = X_ge[train_index], X_ge[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    accuracies_ge.append(acc)

# --- Plot ---
plt.figure(figsize=(8, 5))
plt.boxplot(accuracies_ge)
plt.title("Stratified Random Forest Accuracy (30 Splits)")
plt.ylabel("Accuracy")
plt.xticks([1], ["Random Forest"])
plt.grid(True)
plt.show()

## Merging modalities into one dataset

In [ ]:
# Step 1: Align both dataframes on common patient IDs
common_ids = ge_df_matched.index.intersection(methyl_df_matched.index)
df_expr_aligned = ge_df_matched.loc[common_ids]
df_meth_aligned = methyl_df_matched.loc[common_ids]

# Step 2: Add suffixes to distinguish between modalities (optional but helpful)
df_expr_aligned = df_expr_aligned.add_suffix('_expr')
df_meth_aligned = df_meth_aligned.add_suffix('_meth')

# Step 2: Impute missing values modality-wise for late fusion
imp = SimpleImputer(strategy='mean')
X_expr_imputed = imp.fit_transform(df_expr_aligned) # this is not necessary in our case
X_meth_imputed = imp.fit_transform(df_meth_aligned)

# Step 3: Concatenate the two dataframes along columns
X_merged = pd.concat([df_expr_aligned, df_meth_aligned], axis=1)

# Result: each row = patient, columns = gene_expr + gene_meth features
print(X_merged.shape)
print(X_merged.head())

# Matching the output labels:
y = target_series.loc[common_ids].astype(int)

## Early fusion

In [ ]:
# --- Step 3: Impute missing values ---
imputer = SimpleImputer(strategy='mean')
X_merged_imputed = imputer.fit_transform(X_merged)

sss = StratifiedShuffleSplit(n_splits=30, test_size=0.3, random_state=42)
accuracies_early_fusion = []

for train_index, test_index in sss.split(X_merged_imputed, y):
    X_train, X_test = X_merged_imputed[train_index], X_merged_imputed[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    accuracies_early_fusion.append(acc)

# --- Plot ---
plt.figure(figsize=(8, 5))
plt.boxplot(accuracies_early_fusion)
plt.title("Stratified Random Forest Accuracy (30 Splits)")
plt.ylabel("Accuracy")
plt.xticks([1], ["Random Forest"])
plt.grid(True)
plt.show()

## Late fusion

In [ ]:
sss = StratifiedShuffleSplit(n_splits=30, test_size=0.3, random_state=42)
accuracies_late_fusion = []

for train_index, test_index in sss.split(X_expr_imputed, y):
    X_expr_train, X_expr_test = X_expr_imputed[train_index], X_expr_imputed[test_index]
    X_meth_train, X_meth_test = X_meth_imputed[train_index], X_meth_imputed[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train separate RF models on each modality
    clf_expr = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    clf_meth = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

    clf_expr.fit(X_expr_train, y_train)
    clf_meth.fit(X_meth_train, y_train)

    # Predict probabilities from both
    prob_expr = clf_expr.predict_proba(X_expr_test)
    prob_meth = clf_meth.predict_proba(X_meth_test)

    # Average predicted probabilities (late fusion)
    prob_avg = (prob_expr + prob_meth) / 2.0
    y_pred = np.argmax(prob_avg, axis=1)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracies_late_fusion.append(acc)

# Step 4: Plot results
plt.figure(figsize=(8, 5))
plt.boxplot(accuracies_late_fusion)
plt.title("Random Forest with Late Fusion (30 Stratified Splits)")
plt.ylabel("Accuracy")
plt.xticks([1], ["Late Fusion RF"])
plt.grid(True)
plt.show()

## Plotting all the accuracies on the boxplot. What do you observe? Which modality has stronger signal? Which fusion is more beneficial to use? 

In [ ]:
# Combine them into a list of lists
all_accuracies = [
    accuracies_meth,
    accuracies_ge,
    accuracies_early_fusion,
    accuracies_late_fusion
]

# Define labels for each boxplot
labels = [
    'Methylation Only',
    'Gene Expression Only',
    'Early Fusion',
    'Late Fusion'
]

# Create the plot
plt.figure(figsize=(10, 6))
plt.boxplot(all_accuracies, tick_labels=labels, patch_artist=True)

# Customize appearance
plt.title("Comparison of Model Accuracies")
plt.ylabel("Accuracy")
plt.grid(True)
plt.xticks(rotation=15)

plt.show()